# 아이템 기반 CF

In [2]:
import pandas as pd
import numpy as np

In [3]:
path = "udata.csv"
df = pd.read_csv(path, sep='\t', names =  ['UserID', 'ItemID', 'Rating', 'Timestamp'], header = None)

In [4]:
#사용자 총 수 -> n_users 변수 생성
n_users = df.UserID.unique().shape[0]

In [5]:
#영화 총수 -> n_items 변수 생성
n_items = df['ItemID'].unique().shape[0]

In [6]:
#각 셀에 평가 값을 저장 <- 크기가 n_users*n_items인 배열 값을 모두 0으로 생성
ratings = np.zeros((n_users, n_items))

In [7]:
#df의 각 튜플에서 행의 각 열에 대한 정보를 추출해 평가 행렬의 셀에 저장
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
#테스트 크기 =0.33 / 랜덤시드 =42
ratings_train, ratings_test = train_test_split(ratings, test_size=0.33, random_state=42)

# IBCF 구축 - 영화수 = k

In [10]:
from sklearn.neighbors import NearestNeighbors

In [11]:
k = ratings_train.shape[1]

In [12]:
neigh = NearestNeighbors(k, 'cosine')

In [13]:
neigh.fit(ratings_train.T)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=1682, p=2,
                 radius='cosine')

In [14]:
top_k_distances, top_k_users = neigh.kneighbors(ratings_train.T,
                                              return_distance=True)

In [15]:
top_k_distances.shape

(1682, 1682)

In [16]:
item_pred = ratings_train.dot(top_k_distances) / np.array([np.abs(top_k_distances).sum(axis=1)])

In [17]:
item_pred.shape

(631, 1682)

# 모델 평가하기

In [18]:
import numpy as np
import sklearn

In [19]:
dist_out = 1- sklearn.metrics.pairwise.cosine_distances(ratings_train)

In [20]:
user_pred = dist_out.dot(ratings_train) / np.array([np.abs(dist_out).sum(axis=1)]).T

In [21]:
from sklearn.metrics import mean_squared_error
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten() 
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [22]:
get_mse(item_pred, ratings_train)

11.130000188318895

In [23]:
get_mse(item_pred, ratings_test)

12.128683035513326

# K-최접 이웃에 대한 훈련 모델

In [24]:
k=40

In [25]:
neigh2 = NearestNeighbors(k, 'cosine')

In [26]:
neigh2.fit(ratings_train.T)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=40, p=2,
                 radius='cosine')

In [27]:
top_k_distance, top_k_movies = neigh2.kneighbors(ratings_train.T, return_distance=True)

In [28]:
pred = np.zeros(ratings_train.T.shape)

In [ ]:
for i in range(ratings_train.T.shape[0]):
    pred[i,:] = top_k_distances[i].dot(ratings_train.T[top_k_users][i])/np.array([np.abs(top_k_distances[i]).sum(axis=0)]).T

In [ ]:
pred

In [ ]:
get_mse(item_pred_k, ratings_train)

In [ ]:
get_mse(item_pred_k, ratings_test)